In [ ]:
! wget https://edinburghnlp.inf.ed.ac.uk/workshops/SemEval2022/train.En.csv

--2022-01-11 07:19:22--  https://edinburghnlp.inf.ed.ac.uk/workshops/SemEval2022/train.En.csv
Resolving edinburghnlp.inf.ed.ac.uk (edinburghnlp.inf.ed.ac.uk)... 129.215.33.77, 2001:630:3c1:33:216:3eff:fee2:6fec
Connecting to edinburghnlp.inf.ed.ac.uk (edinburghnlp.inf.ed.ac.uk)|129.215.33.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491939 (480K) [text/csv]
Saving to: ‘train.En.csv’

train.En.csv        100%[===================>] 480.41K   898KB/s    in 0.5s    

2022-01-11 07:19:24 (898 KB/s) - ‘train.En.csv’ saved [491939/491939]



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv('train.En.csv')
df.dropna(subset=['tweet'], inplace=True)
df.fillna('', inplace=True)
df

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring...",0,1,0,0,0,0
1,1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...,1,0,0,0,0,0
2,2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall...",0,1,0,0,0,0
3,3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to...",1,0,0,0,0,0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3463,3463,The population spike in Chicago in 9 months is...,0,,,,,,,
3464,3464,You'd think in the second to last English clas...,0,,,,,,,
3465,3465,I’m finally surfacing after a holiday to Scotl...,0,,,,,,,
3466,3466,Couldn't be prouder today. Well done to every ...,0,,,,,,,


In [ ]:
tweets = df.loc[df['sarcastic'].astype(int) == 1]
tweets = tweets.drop(['sarcastic', 'sarcasm', 'irony', 'satire', 'understatement', 'overstatement', 'rhetorical_question'], axis=1)
tweets['all'] = tweets['tweet']
tweets

,Unnamed: 0,tweet,rephrase,all
0,0,The only thing I got from college is a caffein...,"College is really difficult, expensive, tiring...",The only thing I got from college is a caffein...
1,1,I love it when professors draw a big question ...,I do not like when professors don’t write out ...,I love it when professors draw a big question ...
2,2,Remember the hundred emails from companies whe...,"I, at the bare minimum, wish companies actuall...",Remember the hundred emails from companies whe...
3,3,Today my pop-pop told me I was not “forced” to...,"Today my pop-pop told me I was not ""forced"" to...",Today my pop-pop told me I was not “forced” to...
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,I would say Ted Cruz is an asshole and doesn’t...,@VolphanCarol @littlewhitty @mysticalmanatee I...
...,...,...,...,...
862,862,yo @claires do yall do hysterectomies?,"Claires, you should not do full hysterectomies.",yo @claires do yall do hysterectomies?
863,863,@JacobWohlReport Do I need to aquire a wife be...,A lot of people don't have a wife.,@JacobWohlReport Do I need to aquire a wife be...
864,864,I get a lot of boy who cried wolf vibes from t...,The red cross is always needy.,I get a lot of boy who cried wolf vibes from t...
865,865,Update: holding hands with your mom and walkin...,Holding hands with your parent while walking a...,Update: holding hands with your mom and walkin...


In [ ]:
labels = df.loc[df['sarcastic'] == 1]
labels['categories'] = labels['sarcasm'].astype(int).astype(str) + labels['irony'].astype(int).astype(str) + labels['satire'].astype(int).astype(str) + labels['understatement'].astype(int).astype(str) + labels['overstatement'].astype(int).astype(str) + labels['rhetorical_question'].astype(int).astype(str)
labels['categories']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0      010000
1      100000
2      010000
3      100000
4      100000
        ...  
862    100001
863    100001
864    010000
865    100000
866    100000
Name: categories, Length: 867, dtype: object

In [ ]:
words = tweets.copy()

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import re
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
corpus = []
corpus_test = []

for i in range(len(words)):
    review = re.sub('[^a-zA-Z]', ' ', words['all'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
X = tfidf_vec.fit_transform(corpus).toarray()
X.shape

(867, 5000)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels['categories'], test_size=0.2, random_state=8)

In [ ]:
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

previous_score = 0
for alpha in np.arange(0, 1, 0.1):
    sub_classifier = MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train, y_train)
    y_pred = sub_classifier.predict(X_test)
    score = metrics.accuracy_score(y_test, y_pred)
    
    if score > previous_score:
        classifier = sub_classifier
        previous_score = score

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


In [ ]:
prediction = classifier.predict(X_test)

report = metrics.classification_report(y_test, prediction, output_dict=True)

report

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'010000': {'f1-score': 0.07692307692307693,
  'precision': 0.5,
  'recall': 0.041666666666666664,
  'support': 24},
 '010001': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 5},
 '010010': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 1},
 '100000': {'f1-score': 0.8287671232876712,
  'precision': 0.7076023391812866,
  'recall': 1.0,
  'support': 121},
 '100001': {'f1-score': 0.14285714285714288,
  'precision': 1.0,
  'recall': 0.07692307692307693,
  'support': 13},
 '100010': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 5},
 '100100': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 1},
 '101000': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 3},
 '110000': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 1},
 'accuracy': 0.7068965517241379,
 'macro avg': {'f1-score': 0.11650526034087677,
  'precision': 0.24528914879792074,
  'recall': 0.12428774928774929,
  'support': 174},
 'weighted avg': {'f1